In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!git clone https://github.com/mueedurrehman/distiller.git

Cloning into 'distiller'...
remote: Enumerating objects: 6174, done.
remote: Total 6174 (delta 0), reused 0 (delta 0), pack-reused 6174
Receiving objects: 100% (6174/6174), 39.82 MiB | 34.24 MiB/s, done.
Resolving deltas: 100% (4331/4331), done.


In [17]:
# %cd ..

/content/distiller


# Setup Distiller

In [1]:
%cd distiller

!pip3 install -e .

/content/distiller
Obtaining file:///content/distiller
  Found existing installation: distiller 0.4.0rc0
    Can't uninstall 'distiller'. No files were found to uninstall.
  Running setup.py develop for distiller


In [2]:
!pip install xdelta3

# Compute Statistic

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import distiller
import distiller.models
import xdelta3

import time
import math
import copy
import zipfile
from enum import Enum
import zipfile
import sys
import glob


from collections import OrderedDict
import os
import pandas as pd

In [4]:
def extractIndicesVals(vals):
  matrix = copy.deepcopy(vals)
  matrix = matrix.numpy()
  values = np.ravel(matrix)
  indices = (values != 0).astype(np.int8)
  values = values[values != 0]
  indicesPacked = np.packbits(indices)
  return indicesPacked, values

def computeStorageTensor(name, tensor,zipfile):
  indicesPacked, values = extractIndicesVals(tensor)
  totalSize = 0
  np.save(name + "I.npy", indicesPacked)
  np.save(name + "V.npy", values)
  zipfile.write(name + "I.npy")
  zipfile.write(name + "V.npy")
  return totalSize

def deltaEncodeIndicesValues(indices, values):
  indicesDelta = []
  valsDelta = []
  if indices[0] > 8:
    numZeros = indices[0] // 8
    for i in range(numZeros):
      valsDelta.append(0)
      #append a 7, which corresponds to a delta of 8
      indicesDelta.append(7)
    indicesDelta.append((indices[0] % 8) - 1)
    #now add the value at this index
    valsDelta.append(values[0])
  #first value has been handled, compute differences and handle those
  indices = np.diff(indices)
  for i in range(len(indices)):
    index = indices[i]
    if index > 8:
      numZeros = index // 8
      for j in range(numZeros):
        valsDelta.append(0)
        #append a 7, which corresponds to a delta of 8
        indicesDelta.append(7)
      indicesDelta.append((index % 8) - 1)
      #now add the value at this index
      valsDelta.append(values[i + 1])
    else:
      indicesDelta.append(index - 1)
      valsDelta.append(values[i + 1])
  return np.array(indicesDelta, dtype = np.int8), np.array(valsDelta, dtype = np.float32)

def convertDeltaIndicesToBits(deltaIndices):
  position = np.ones(deltaIndices.shape, dtype = np.int8) * 4
  result = None
  for i in range(3):
    bits = np.bitwise_and(deltaIndices, position)
    bits = np.right_shift(bits, 2 - i)
    if result is None:
      result = bits.reshape(1,-1)
    else:
      result = np.concatenate((result,bits.reshape(1,-1)), axis = 0)
    position = np.right_shift(position, 1)
  # print("the length of indices in encoding is", np.ravel(result, order = 'F').shape)
  return np.packbits(np.ravel(result, order = 'F'))

def extractDeltaIndicesVals(vals):
  matrix = copy.deepcopy(vals)
  matrix = matrix.numpy()
  values = np.ravel(matrix)
  indices = np.argwhere(values != 0).flatten()
  values = values[values != 0]
  return deltaEncodeIndicesValues(indices, values)

def computeStorageTensorDelta(name, tensor, zipfile):
  indices, values = extractDeltaIndicesVals(tensor)
  indices = convertDeltaIndicesToBits(indices)
  totalSize = 0
  np.save(name + "I.npy", indices)
  np.save(name + "V.npy", values)
  zipfile.write(name + "I.npy")
  zipfile.write(name + "V.npy")

def processDictionary(dictionary, computeStorageFunction):
  totalSize = 0
  newDictionary = OrderedDict()
  zipTypes = [zipfile.ZIP_LZMA, zipfile.ZIP_DEFLATED]
  sizes = []
  for zipType in zipTypes:
    if not os.path.exists('archive.zip'):
      os.mknod('archive.zip')
    deflated = zipfile.ZipFile("archive.zip", mode = 'w', compression = zipType)
    for key,value in dictionary.items():
      newDictionary[key] = value.size()
      computeStorageFunction(key, value.cpu(), deflated)
    torch.save(newDictionary, "dictionaryTorch")
    deflated.write("dictionaryTorch")
    sizes.append(os.path.getsize("archive.zip"))
    os.remove("archive.zip")
  return sizes

def compressDictionary(dictionary, method):
  if method == "bitmap":
    return processDictionary(dictionary, computeStorageTensor)
  elif method == "deltaEncoding":
    return processDictionary(dictionary, computeStorageTensorDelta)

In [5]:
#helper functions

transformTest = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = transformTest)
testLoader = torch.utils.data.DataLoader(test, batch_size=256,
                                         shuffle=False, num_workers=2)
def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

def computeStatePyTorch(state_dict, compressed = True):
  originalSize = 0
  compressedSize = 0
  torch.save(state_dict, "./curStateDict")
  totalSize = os.path.getsize("./curStateDict")
  if compressed:
    for value in state_dict.values():
      torch.save(value, "denseTensor")
      sparse = to_sparse(value)
      torch.save(sparse, "sparse")
      originalSize += os.path.getsize("./denseTensor")
      compressedSize += os.path.getsize("./sparse")
    os.remove("./sparse")
    return totalSize - originalSize + compressedSize

def computeSize(stateDictionary):
  torch.save(stateDictionary, "stateDict")
  size = os.path.getsize("stateDict")
  os.remove("stateDict")
  return size

def computeTotalParameters(model):
  return sum(p.numel() for p in model.parameters())

def computeNonZeroParameters(model):
  return sum(p.nonzero().size(0) for p in model.parameters())

baseModelCheckpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Baseline Models/baseResnet56")
baseModel = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
baseModel.load_state_dict(baseModelCheckpoint["state_dict"])
baseParameters = computeTotalParameters(baseModel)
baseSize = computeSize(baseModel.state_dict())
# _, baseTime = test(baseModel)

def computeIndexSavingRate(baseParameters, compressedParameters):
  return (baseParameters - compressedParameters)/compressedParameters

#accuracy will not be accurate for the quantized models
def test(net):
    startTime = time.time()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testLoader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    return acc, time.time() - startTime

def computeStatistics(compressedModel):
  modelParameters = computeNonZeroParameters(compressedModel)
  indexSavingRate = computeIndexSavingRate(baseParameters, modelParameters)
  parameterCompressionRate = baseParameters / modelParameters
  accuracy,time = test(compressedModel)
  accuracy = round(accuracy, 2)
  bitmapSizes = compressDictionary(compressedModel.state_dict(), "bitmap")
  deltaEncodingSizes = compressDictionary(compressedModel.state_dict(), "deltaEncoding")
  size_bitmap_LZMA = bitmapSizes[0]
  size_bitmap_DEFLATED = bitmapSizes[1]
  size_deltaEncoding_LZMA = deltaEncodingSizes[0]
  size_deltaEncoding_DEFLATED = deltaEncodingSizes[1]
  scr_bitmap_LZMA = baseSize / size_bitmap_LZMA
  scr_bitmap_DEFLATED = baseSize / size_bitmap_DEFLATED
  scr_deltaEncoding_LZMA = baseSize / size_deltaEncoding_LZMA
  scr_deltaEncoding_DEFLATED = baseSize / size_deltaEncoding_DEFLATED
 
  sparsity = round(distiller.utils.model_sparsity(compressedModel), 2)
  #bitmap sizes have 2 sizes, deltaEncoding has 2 sizes, first size is LZMA, second size is deflated, this means 4 size compression rates are computed
  #Add to return value 4 size compression rates as well as original sizes under each scheme
  # Returning 4 size compression rates, 4 sizes, accuracy, time, parameterCompressionRate, IndexSavingRate, modelParameters, sparsity
  # return {"Size Compression Rate" : sizeCompressionRate, "ParameterCompressionRate" : parameterCompressionRate, "IndexSavingRate": indexSavingRate}

  return {"Accuracy" : accuracy,
          "Sparsity" : sparsity,
          "Time" : time,
          "Model Parameters" : modelParameters, 
          "Size_bitmap_LZMA" : size_bitmap_LZMA,
          "Size_bitmap_DEFLATED" : size_bitmap_DEFLATED,
          "Size_deltaEncoding_LZMA" : size_deltaEncoding_LZMA,
          "Size_deltaEncoding_DEFLATED" : size_deltaEncoding_DEFLATED,
          "SCR_bitmap_LZMA" : scr_bitmap_LZMA,
          "SCR_bitmap_DEFLATED" : scr_bitmap_DEFLATED,
          "SCR_deltaEncoding_LZMA" : scr_deltaEncoding_LZMA,
          "SCR_deltaEncoding_DEFLATED" : scr_deltaEncoding_DEFLATED,          
          "ParameterCompressionRate" : parameterCompressionRate, 
          "IndexSavingRate": indexSavingRate 
  }
          # "Train Parameter Compression Rate" : parameterCompressionRateTrain, 
          # "Train Index Saving Rate" : indexSavingRateTrain}

  0%|          | 0/170498071 [00:00<?, ?it/s]

170500096it [00:01, 96903173.10it/s]                                


Extracting ./data/cifar-10-python.tar.gz to ./data


In [6]:
# Sun Dec 13 06:56:50 2020       
# +-----------------------------------------------------------------------------+
# | NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
# |-------------------------------+----------------------+----------------------+
# | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
# | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
# |                               |                      |               MIG M. |
# |===============================+======================+======================|
# |   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
# | N/A   76C    P0    34W /  70W |   1121MiB / 15079MiB |      0%      Default |
# |                               |                      |                 ERR! |
# +-------------------------------+----------------------+----------------------+
                                                                               
# +-----------------------------------------------------------------------------+
# | Processes:                                                                  |
# |  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
# |        ID   ID                                                   Usage      |
# |=============================================================================|
# |  No running processes found                                                 |
# +-----------------------------------------------------------------------------+
!nvidia-smi

Fri Dec 18 23:40:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |    811MiB / 15079MiB |      5%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
originalModel = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
originalKeys = set()

for key in originalModel.state_dict():
    originalKeys.add(key)

del originalModel

def filterWeights(quantType, key):
  # return True
  if quantType == "QA":
    return filterWeightsQA(key)
  elif quantType == "Post":
    return filterWeightsPost(key)

def filterWeightsQA(key):
  if key in originalKeys:
    return True
  else:
      if "weight_scale" in key:
        return False
      elif "weight_zero_point" in key:
        return False
      elif "float_weight" in key:
        return False
      elif "relu" in key:
        if "biased" in key or "ema_decay" in key or "iter_count" in key :
          return False
      return True

def filterWeightsPost(key):
  if key in originalKeys:
    return True
  else:
    if "num_forwards" in key or "force_readjust" in key or "w_scale" in key or "fp_bias" in key:
      return False
    return True

def canQuantizeLayer(quantType, name):
  if quantType == "QA":
    if "conv" in name or "fc" in name or "relu" in name:
      return True
    return False
  elif quantType == "Post":
    if "conv" in name or "fc" in name or "eltwise" in name or "matmul" in name or "concat" in name:
      return True
    return False

def extractIndicesValsPacked(vals):
  matrix = copy.deepcopy(vals)
  matrix = matrix.numpy()
  values = np.ravel(matrix)
  indices = (values != 0).astype(np.int8)
  values = values[values != 0].astype(np.float32)
  indicesPacked = np.packbits(indices)
  return indicesPacked, values

def allZeros(array):
  return not np.any(array)

def computeStorageTensor(name, tensor,zipfile, quantType, bits = 32):
  indicesPacked, values = extractIndicesValsPacked(tensor)
  if bits != 32 and canQuantizeLayer(quantType, name):
    values = compressBits(values, bits)
  if not allZeros(values):
    np.save(name + "I.npy", indicesPacked)
    np.save(name + "V.npy", values)
    zipfile.write(name + "I.npy")
    zipfile.write(name + "V.npy")

def deltaEncodeIndicesValues(indices, values):
  indicesDelta = []
  valsDelta = []
  if indices[0] > 8:
    numZeros = indices[0] // 8
    for i in range(numZeros):
      valsDelta.append(0)
      #append a 7, which corresponds to a delta of 8
      indicesDelta.append(7)
    indicesDelta.append((indices[0] % 8) - 1)
    #now add the value at this index
    valsDelta.append(values[0])
  #first value has been handled, compute differences and handle those
  indices = np.diff(indices)
  for i in range(len(indices)):
    index = indices[i]
    if index > 8:
      numZeros = index // 8
      for j in range(numZeros):
        valsDelta.append(0)
        #append a 7, which corresponds to a delta of 8
        indicesDelta.append(7)
      indicesDelta.append((index % 8) - 1)
      #now add the value at this index
      valsDelta.append(values[i + 1])
    else:
      indicesDelta.append(index - 1)
      valsDelta.append(values[i + 1])
  return np.array(indicesDelta, dtype = np.int32), np.array(valsDelta, dtype = np.float32)

def extractDeltaIndicesVals(vals):
  matrix = copy.deepcopy(vals)
  matrix = matrix.numpy()
  values = np.ravel(matrix)
  indices = np.argwhere(values != 0).flatten()
  values = values[values != 0]
  return indices, values

# def extractFloatingBits(values, bitsToRead):
#   for value in values:
#     f1 = bitstring.BitArray(float=1.5, length=32)
#     print(type(f1.bin))
#     print(f1.bin)
#     print(int(f1.bin, 2))

def randomIntegers(values, bitsAvailable):
  a = np.random.randn(values.size) * (2**(bitsAvailable - 1))
  return a.astype(np.int32)

def compressBits(values, bitsToRead):
  #simulate conversion of floating point to integers
  if values.dtype == np.float32:
    values = randomIntegers(values, bitsToRead)
    # values = values.astype(np.int32)
  position = np.ones(values.shape, dtype = np.int32)
  position = np.left_shift(position, bitsToRead - 1)
  result = None
  for i in range(bitsToRead):
    # print(type(values), type(position), i)
    # print(values.dtype, position.dtype)
    bits = np.bitwise_and(values, position)
    bits = np.right_shift(bits, bitsToRead - 1 - i)
    if result is None:
      result = bits.reshape(1,-1)
    else:
      result = np.concatenate((result,bits.reshape(1,-1)), axis = 0)
    position = np.right_shift(position, 1)
  # return np.ravel(result, order = 'F')
  return np.packbits(np.ravel(result, order = 'F'))

def computeStorageTensorDelta(name, tensor, zipfile, quantType, bits = 32):
  indices, values = extractDeltaIndicesVals(tensor)
  if not allZeros(values):
    indices, values = deltaEncodeIndicesValues(indices, values)
    if bits != 32 and canQuantizeLayer(quantType, name):
      values = compressBits(values, bits)
    indices = compressBits(indices, 3)
    np.save(name + "I.npy", indices)
    np.save(name + "V.npy", values)
    zipfile.write(name + "I.npy")
    zipfile.write(name + "V.npy")

def extractValues(tensor):
  matrix = copy.deepcopy(tensor)
  matrix = matrix.numpy()
  values = np.ravel(matrix).astype(np.float32)
  return values

def computeStorageTensorBitArray(name, tensor, zipfile, quantType, bits = 32):
  values = extractValues(tensor)
  if not allZeros(values):
    if bits != 32 and canQuantizeLayer(quantType, name):
      values = compressBits(values, bits)
    np.save(name + "V.npy", values)
    zipfile.write(name + "V.npy")

def processDictionary(dictionary, computeStorageFunction, quantType = None, bits = 32):
  totalSize = 0
  newDictionary = OrderedDict()
  zipTypes = [zipfile.ZIP_LZMA, zipfile.ZIP_DEFLATED]
  sizes = []
  for zipType in zipTypes:
    if not os.path.exists('archive.zip'):
      os.mknod('archive.zip')
    deflated = zipfile.ZipFile("archive.zip", mode = 'w', compression = zipType)
    for key,value in dictionary.items():
      #only look at layers that matter for inference
      if quantType is None or filterWeights(quantType, key):
        newDictionary[key] = value.size()
        computeStorageFunction(key, value.cpu(), deflated, quantType, bits)
    torch.save(newDictionary, "dictionaryTorch")
    deflated.write("dictionaryTorch")
    sizes.append(os.path.getsize("archive.zip"))
    os.remove("archive.zip")
  return sizes

# def computeStorageTensorBitArray(name, tensor, zipfile, quantType, bits = 32):
#   values = extractValues(tensor)
#   if not allZeros(values):
#     if bits != 32 and canQuantizeLayer(quantType, name):
#       values = compressBits(values, bits)
#     np.save(name + "V.npy", values)
#     return os.path.getsize(name + "V.npy")
#   return 0
    # zipfile.write(name + "V.npy")

# def processDictionary(dictionary, computeStorageFunction, quantType = None, bits = 32):
#   totalSize = 0
#   newDictionary = OrderedDict()
#   for key,value in dictionary.items():
#     #only look at layers that matter for inference
#     if quantType is None or filterWeights(quantType, key):
#       newDictionary[key] = value.size()
#       totalSize += computeStorageFunction(key, value.cpu(), None, quantType, bits)
#   torch.save(newDictionary, "dictionaryTorch")
#   totalSize += os.path.getsize("dictionaryTorch")
#   return totalSize

def computeSizeZip(stateDictionary):
  zipTypes = [zipfile.ZIP_LZMA, zipfile.ZIP_DEFLATED]
  sizes = []
  for ziptype in zipTypes:
    if not os.path.exists('archiveBase.zip'):
      os.mknod('archiveBase.zip')
    deflated = zipfile.ZipFile("archiveBase.zip", mode = 'w', compression = ziptype)
    torch.save(stateDictionary, "stateDict")
    deflated.write("stateDict")
    sizes.append(os.path.getsize("archiveBase.zip"))
    os.remove("archiveBase.zip")
  return sizes

#Post or QA for quantType
def compressDictionary(dictionary, method, quantType, bits = 32):
  if method == "bitmap":
    return processDictionary(dictionary, computeStorageTensor, quantType, bits)
  elif method == "deltaEncoding":
    return processDictionary(dictionary, computeStorageTensorDelta, quantType, bits)
  elif method == "bitArray":
    return processDictionary(dictionary, computeStorageTensorBitArray, quantType, bits)
  elif method == "zipOnly":
    return computeSizeZip(dictionary)

# def computeSize(stateDictionary):
#   torch.save(stateDictionary, "stateDict")
#   size = os.path.getsize("stateDict")
#   os.remove("stateDict")
#   return size

def computeSize(stateDictionary):
  torch.save(stateDictionary, "stateDict")
  size = os.path.getsize("stateDict")
  os.remove("stateDict")
  return size

#USE THIS MOVING FORWARD TO COMPUTE A MODEL's Total Parameters
def computeNonZeroParametersDictionary(stateDictionary, quantType):
  return sum(p.nonzero().size(0) for key,p in stateDictionary.items() if filterWeights(quantType, key))

def computeTotalParameters(model):
  return sum(p.numel() for p in model.parameters())

baseModelCheckpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Baseline Models/baseResnet56")
baseModel = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
baseModel.load_state_dict(baseModelCheckpoint["state_dict"])
baseParameters = computeTotalParameters(baseModel)
baseSize = computeSize(baseModel.state_dict())

# model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
# baseModel.load_state_dict(baseModelCheckpoint["state_dict"])
# baseParameters = computeTotalParametersDictionary(model.state_dict())
# print(baseParameters)

def computeIndexSavingRate(baseParameters, compressedParameters):
  return (baseParameters - compressedParameters)/compressedParameters

In [11]:
def computeSomeStatistics(checkpoint):
  modelParameters = computeNonZeroParametersDictionary(checkpoint["state_dict"], quant)
  parameterCompressionRate = baseParameters / modelParameters
  indexSavingRate = computeIndexSavingRate(baseParameters, modelParameters)
  bitM = compressDictionary(checkpoint["state_dict"], "bitmap", quant, bits)
  delta = compressDictionary(checkpoint["state_dict"], "deltaEncoding", quant, bits)
  bitA = compressDictionary(checkpoint["state_dict"], "bitArray", quant, bits)
  zipO = compressDictionary(checkpoint["state_dict"], "zipOnly", quant, bits)      
  scr_bitmap_LZMA = baseSize/bitM[0]
  scr_bitmap_DEFLATED = baseSize/bitM[1]
  scr_deltaEncoding_LZMA = baseSize/delta[0]
  scr_deltaEncoding_DEFLATED = baseSize/delta[1]  
  scr_bitarray_LZMA = baseSize/bitA[0]
  scr_bitarray_DEFLATED = baseSize/bitA[1]
  scr_zip_LZMA = baseSize/zipO[0]
  scr_zip_DEFLATED = baseSize/zipO[1]   
  stats_rest = {"Model Parameters" : modelParameters,
                "Size_bitmap_LZMA" : bitM[0],
                "Size_bitmap_DEFLATED" : bitM[0],
                "Size_deltaEncoding_LZMA" : delta[0],
                "Size_deltaEncoding_DEFLATED" : delta[1],
                "Size_bitarray_LZMA" : bitA[0],
                "Size_bitarray_DEFLATED" : bitA[1],
                "Size_zip_LZMA" : zipO[0],
                "Size_zip_DEFLATED" : zipO[1],      
                "SCR_bitmap_LZMA" : scr_bitmap_LZMA,
                "SCR_bitmap_DEFLATED" : scr_bitmap_DEFLATED,
                "SCR_deltaEncoding_LZMA" : scr_deltaEncoding_LZMA,
                "SCR_deltaEncoding_DEFLATED" : scr_deltaEncoding_DEFLATED, 
                "scr_bitarray_LZMA": scr_bitarray_LZMA,
                "scr_bitarray_DEFLATED": scr_bitarray_DEFLATED,
                "scr_zip_LZMA": scr_zip_LZMA,
                "scr_zip_DEFLATED": scr_zip_DEFLATED,
                "ParameterCompressionRate" : parameterCompressionRate, 
                "IndexSavingRate": indexSavingRate 
  }
  stats.update(stats_rest)
  return stats

In [14]:
# Structure Pruning
pd.options.display.float_format = '{:.2f}'.format

modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Pruning Models/Structure Pruning/ActivationAPoZRankedFilterPruner"

#MUST ENSURE YOU HAVE THE SAME GPU BEFORE RUNNING
# round to 2 decimal places
df = pd.DataFrame()
for file in os.listdir(modelDirectory):
  target = os.path.join(modelDirectory, file, "best.pth.tar")
  print(file)
  if os.path.exists(target):
      # model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
      #load the file
      checkpoint = torch.load(target)
      # model.load_state_dict(checkpoint["state_dict"])
      stats = computeSomeStatistics(checkpoint)   
      stats['Accur-Spar'] =  str(stats["Accuracy"]) + "-" + str(stats['Sparsity'])         
      df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
      df = pd.concat([df, df_temp], axis=1)
      print()
df = df.T
df



Averagepercentage of zeros (APoZ) V2


NameError: ignored

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Pruning Models/Structure Pruning/ActivationAPoZRankedFilterPruner"

#MUST ENSURE YOU HAVE THE SAME GPU BEFORE RUNNING
# round to 2 decimal places
df = pd.DataFrame()
for file in os.listdir(modelDirectory):
  target = os.path.join(modelDirectory, file, "best.pth.tar")
  print(file)
  if os.path.exists(target):
      model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
      #load the file
      checkpoint = torch.load(target)
      model.load_state_dict(checkpoint["state_dict"])
      stats = computeStatistics(model)   
      stats['Accur-Spar'] =  str(stats["Accuracy"]) + "-" + str(stats['Sparsity'])         
      df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
      df = pd.concat([df, df_temp], axis=1)
      print()
df = df.T
df



In [ ]:
pd.options.display.float_format = '{:.2f}'.format

modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/IterativeSensitivity"

#MUST ENSURE YOU HAVE THE SAME GPU BEFORE RUNNING
# round to 2 decimal places
df = pd.DataFrame()
for file in os.listdir(modelDirectory):
  target = os.path.join(modelDirectory, file, "best.pth.tar")
  print(file)
  if os.path.exists(target):
      model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
      #load the file
      checkpoint = torch.load(target)
      model.load_state_dict(checkpoint["state_dict"])
      stats = computeStatistics(model)   
      stats['Accur-Spar'] =  str(stats["Accuracy"]) + "-" + str(stats['Sparsity'])         
      df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
      df = pd.concat([df, df_temp], axis=1)
      print()
df = df.T
df

In [ ]:
# microfade before beep
# chaing english ending to giberish (backgward text)
# lowpass earthquake,
# schurb tool before the english
# reverb on train 
# Command e, add a new monotrack, hold down options, duplicate and you can draw over section 
# select, audiosuite, gain, and make gain to very small 0 to english

In [ ]:
df.T

,Accuracy,Sparsity,Time,Model Parameters,Size_bitmap_LZMA,Size_bitmap_DEFLATED,Size_deltaEncoding_LZMA,Size_deltaEncoding_DEFLATED,SCR_bitmap_LZMA,SCR_bitmap_DEFLATED,SCR_deltaEncoding_LZMA,SCR_deltaEncoding_DEFLATED,ParameterCompressionRate,IndexSavingRate,Accur-Spar
2020.12.03-073428,10.00,100.00,2.82,855506,113913,105897,114413,106294,30.78,33.10,30.64,32.98,1.00,0.00,10.0-100.0
2020.12.03-080635,75.99,62.16,2.86,326494,1426748,1425424,1463509,1463460,2.46,2.46,2.40,2.40,2.62,1.62,75.99-62.16
2020.12.03-082420,10.00,100.00,2.80,855770,113595,105583,114086,105980,30.86,33.20,30.73,33.08,1.00,0.00,10.0-100.0
2020.12.03-082542,10.00,99.99,2.81,855494,114112,106095,114596,106487,30.72,33.04,30.59,32.92,1.00,0.00,10.0-99.99
2020.12.03-090147,10.00,99.99,2.79,855471,114140,106125,114625,106514,30.71,33.03,30.58,32.91,1.00,0.00,10.0-99.99
2020.12.03-090213,10.00,99.99,2.78,855711,113872,105873,114359,106265,30.79,33.11,30.66,32.99,1.00,0.00,10.0-99.99
2020.12.03-093914,10.00,100.00,2.77,855770,113652,105623,114086,105963,30.85,33.19,30.73,33.08,1.00,0.00,10.0-100.0


In [ ]:
df.T.to_csv("data.csv")
# df.to_csv("/content/drive/MyDrive/CS6787 - Final Project/OneShotElementBaseNoOptResults.csv")

In [ ]:
subfolders

['/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/.ipynb_checkpoints',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotExpLR',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotExp40Epoch',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotElementBaseNoOptResults',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/OneShotElementBaseOptResults',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/TestSchedules',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/IterativeSensitivity',
 '/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/IterativeSensitivitySchedules',
 '/content/drive/MyDrive/CS6787 -

In [ ]:
pd.options.display.float_format = '{:.4f}'.format

modelDirectory = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules"
#MUST ENSURE YOU HAVE THE SAME GPU BEFORE RUNNING

subfolders = [f.name for f in os.scandir(modelDirectory) if f.is_dir()]

for experiments in subfolders:
  path_experiment = os.path.join(modelDirectory, experiments) 
  print(experiments)
  if os.path.exists(path_experiment):
    df = pd.DataFrame()
    for file in os.listdir(path_experiment):
      print(file)
      target = os.path.join(path_experiment, file, "best.pth.tar")
      if os.path.exists(target):
          print(":)")
          model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
          #load the file
          checkpoint = torch.load(target)
          model.load_state_dict(checkpoint["state_dict"])
          stats = computeStatistics(model)
          df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
          df = pd.concat([df, df_temp], axis=1)
          # print()
          stats = computeStatistics(model)   
          stats['Accur-Spar'] =  str(stats["Accuracy"]) + "-" + str(stats['Sparsity'])         
          df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = [file])
          df = pd.concat([df, df_temp], axis=1)
      print()
    df = df.T
    df.to_csv("/content/drive/MyDrive/CS6787 - Final Project/" + experiments + ".csv")

.ipynb_checkpoints
OneShotExpLR
threshold0-1ExpLR.yaml

threshold0ExpLR.yaml

threshold0-5ExpLR.yaml

threshold1ExpLR.yaml

OneShotExp40Epoch
threshold0-1ExpLR.yaml

threshold0-5ExpLR.yaml

threshold0ExpLR.yaml

threshold1ExpLR.yaml

OneShotElementBaseNoOptResults
90.85-52.2
:)

91.95-52.2
:)

89.35-85.94
:)

90.17-85.94
:)

90.02-82.6
:)

91.0-67.71
:)

91.3-67.71
:)

92.73-52.2
:)

92.83-52.2
:)

89.56-85.94
:)

91.24-85.94
:)

90.37-82.6
:)

91.51-82.6
:)

92.21-67.71
:)

92.68-67.71
:)

90.82-82.6
:)

93.0-52.2
:)

92.96-67.71
:)

92.15-82.6
:)

91.99-85.94
:)

92.99-52.2
:)

92.65-67.71
:)

91.5-82.6
:)

91.17-85.94
:)

93.12-52.2
:)

92.87-67.71
:)

92.49-82.6
:)

92.2-85.94
:)

93.07-52.2
:)

92.91-52.2
:)

93.23-52.2
:)

92.14-82.6
:)

91.6-85.94
:)

Results-20201202T035922Z-001.zip

mappingFinal.csv

OneShotElementBaseOptResults
90.98-52.46
:)

91.45-52.46
:)

88.57-86.37
:)

90.37-86.37
:)

89.85-83.01
:)

90.95-83.01
:)

90.96-68.05
:)

91.49-68.05
:)

92.55-52.46
:)

92.97-

In [ ]:
folder = "/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/ComputeStatistics/Pruning/"
writer = pd.ExcelWriter('/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/ComputeStatistics/Pruning/Pruning.xlsx')

extension = 'csv'
all_filename_paths = [i for i in glob.glob(folder +'*.{}'.format(extension))]
for csv_path in all_filename_paths:
    csvfilename = csv_path.replace(folder,'')
    print(csvfilename)
    df = pd.read_csv(csv_path)
    if df.empty == True:
      print('DataFrame is empty')
    else:
      df.to_excel(writer,sheet_name=os.path.splitext(csvfilename)[0])
writer.save()

OneShotExpLR.csv
DataFrame is empty
OneShotExp40Epoch.csv
DataFrame is empty
OneShotElementBaseNoOptResults.csv
TestSchedules.csv
DataFrame is empty
OneShotElementBaseOptResults.csv
IterativeSensitivitySchedules.csv
DataFrame is empty
IterativeSensitivity.csv
SensitivitySchedules0.68Base.csv
DataFrame is empty
SensitivityExperiments1.csv
SensitivityExperiments2.csv
DataFrame is empty
FixedSensitivityExperiments.csv
MSMG Results.csv
DataFrame is empty
MSMG Schedules.csv
DataFrame is empty
MSMGResults.csv
RetrainResults1.csv
TuningExperiments.csv


# Testing

In [ ]:
#testing storage functions
checkpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/TuningExperiments/2020.12.05-184050/best.pth.tar", map_location = "cpu")
# checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Pruning Schedules/FixedSensitivityExperiments/2020.12.05-044938/best.pth.tar", map_location = "cpu")
# checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Schedules/Pruning Schedules/FixedSensitivityExperiments/2020.12.04-142239/best.pth.tar", map_location = "cpu")

print(compressDictionary(checkpoint["state_dict"], "bitmap"))
print(compressDictionary(checkpoint["state_dict"], "deltaEncoding"))

# print(computeStatePyTorch(checkpoint["state_dict"]))

# tensor = checkpoint["state_dict"]["module.conv1.weight"].cpu()
# print(computeStorageTensor("test", tensor))
# print(computeStorageTensorDelta("test", tensor))
# print(np.diff([5,1,2,3]))
# print(computeStorageTensor("test", tensor))
# print(deltaEncodeIndicesValues([2,10,35,42,60], [1,2,3,4,5]))
# print(convertDeltaIndicesToBits(np.array([1,2,3,4,5,6,7])))
# print(np.bitwise_and(2,2))
# print(2&2)
# torch.save(tensor, "original")
# print(os.path.getsize("original"))

[557617, 553397]
[588565, 591048]


In [ ]:
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
checkpoint = torch.load("/content/drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Schedules/Pruning Schedules/TuningExperiments/2020.12.05-184050/best.pth.tar", map_location = "cpu")
model.load_state_dict(checkpoint["state_dict"])
stats = computeStatistics(model)

In [ ]:
stats

{'Accuracy': 92.63,
 'IndexSavingRate': 7.274465060963228,
 'Model Parameters': 103423,
 'ParameterCompressionRate': 8.274465060963228,
 'SCR_bitmap_DEFLATED': 6.334840991187159,
 'SCR_bitmap_LZMA': 6.286899430971437,
 'SCR_deltaEncoding_DEFLATED': 5.931298304029453,
 'SCR_deltaEncoding_LZMA': 5.956320882145557,
 'Size_bitmap_DEFLATED': 553397,
 'Size_bitmap_LZMA': 557617,
 'Size_deltaEncoding_DEFLATED': 591048,
 'Size_deltaEncoding_LZMA': 588565,
 'Sparsity': 88.35507525507808,
 'Time': 2.8572661876678467}

In [ ]:
df_temp = pd.DataFrame.from_dict(stats, orient = 'index', columns = ["Model"])
df_temp

,Model
Accuracy,92.630000
Sparsity,88.355075
Time,2.857266
Model Parameters,103423.000000
Size_bitmap_LZMA,557617.000000
Size_bitmap_DEFLATED,553397.000000
Size_deltaEncoding_LZMA,588565.000000
Size_deltaEncoding_DEFLATED,591048.000000
SCR_bitmap_LZMA,6.286899
SCR_bitmap_DEFLATED,6.334841


In [ ]:
df = pd.DataFrame()
# df = df.append(stats, ignore_index=True, row = ["ModelA"])
df = pd.concat([df, df_temp], axis=1)
df

,Model
Accuracy,92.630000
Sparsity,88.355075
Time,2.857266
Model Parameters,103423.000000
Size_bitmap_LZMA,557617.000000
Size_bitmap_DEFLATED,553397.000000
Size_deltaEncoding_LZMA,588565.000000
Size_deltaEncoding_DEFLATED,591048.000000
SCR_bitmap_LZMA,6.286899
SCR_bitmap_DEFLATED,6.334841


# Ignore

In [ ]:
#helper functions

transformTest = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform = transformTest)
testLoader = torch.utils.data.DataLoader(test, batch_size=256,
                                         shuffle=False, num_workers=2)
                                         

baseModelCheckpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/Baseline Models/baseResnet56") 
baseModel = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
baseModel.load_state_dict(baseModelCheckpoint["state_dict"])

# compute parameters for base model
# TODO: USE DISTILLER FOR COMPUTING SPARSITY. distiller.utils.model_sparsity(model)
# the below also gives correct counts
# def computeTotalParameters(model): 
#   return sum(p.numel() for p in model.parameters() if p.dim() in [2,4])

def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

def computeStateSizes(state_dict, compressed = True):
  originalSize = 0
  compressedSize = 0
  torch.save(state_dict, "./curStateDict")
  totalSize = os.path.getsize("./curStateDict")
  if compressed:
    for value in state_dict.values():
      torch.save(value, "denseTensor")
      sparse = to_sparse(value)
      torch.save(sparse.indices(), "sparseTensorIndices")
      torch.save(sparse.values(), "sparseTensorValues")
      torch.save(sparse.size(), "sparseTensorSize")
      originalSize += os.path.getsize("./denseTensor")
      compressedSize += os.path.getsize("./sparseTensorIndices") + os.path.getsize("./sparseTensorValues") + os.path.getsize("./sparseTensorSize")
    os.remove("./denseTensor")
    os.remove("./sparseTensorIndices")
    os.remove("./sparseTensorValues")
    os.remove("./sparseTensorSize")
    return (totalSize - originalSize + compressedSize) / (1024*1024)
  else:
    return totalSize


# TODO: Use this for computing sparsity instead
# def computeTotalParameters(model):
#   return sum(p.numel() for p in model.parameters() if p.requires_grad)

# def computeZeroParameters(model):
#   return sum(p.numel() - p.nonzero().size(0) for p in model.parameters() if p.requires_grad)

# def computeSparsity(model):
#   return round(computeZeroParameters (model) / computeTotalParameters(model) * 100, 3)

def computeSize(model):
  torch.save(model, "./curModel")
  size = os.path.getsize("./curModel")
  os.remove("./curModel")
  return size / (1024*1024)

def computeCompressedModelSize(model):
  stateSize = computeStateSizes(model.state_dict(), True)
  return stateSize + computeSize(model) - computeStateSizes(model.state_dict(), False)

def computeTotalParameters(model):
  return sum(p.nonzero().size(0) for p in model.parameters())

def computeTotalTrainableParameters(model):
  return sum(p.nonzero().size(0) for p in model.parameters() if p.requires_grad)

baseParameters = computeTotalParameters(baseModel)
baseTrainParameters = computeTotalTrainableParameters(baseModel)
baseSize = computeSize(baseModel)

def computeIndexSavingRate(baseParameters, compressedParameters):
  return (baseParameters - compressedParameters)/compressedParameters

def computeStatistics(compressedModel):
  modelParameters = computeTotalParameters(compressedModel)
  modelTrainParameters = computeTotalTrainableParameters(compressedModel)
  modelSize = computeCompressedModelSize(compressedModel)
  indexSavingRate = computeIndexSavingRate(baseParameters, modelParameters)
  indexSavingRateTrain = computeIndexSavingRate(baseTrainParameters, modelTrainParameters)
  parameterCompressionRate = baseParameters / modelparameters
  parameterCompressionRateTrain =  baseTrainParameters / modelTrainParameters
  size = computeSize(compressedModel)
  sizeCompressionRate = baseSize / size
  sparas_stat = round(distiller.utils.model_sparsity((compressedModel)), 2)
  return {"Accuracy" : accuracy_stat,
          "Sparsity" : sparas_stat,
          "SCR" : scr_
          "Size Compression Rate" : sizeCompressionRate, 
          "ParameterCompressionRate" : parameterCompressionRate, 
          "IndexSavingRate": indexSavingRate, 
          "Train Parameter Compression Rate" : parameterCompressionRateTrain, 
          "Train Index Saving Rate" : indexSavingRateTrain}

def test(net):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testLoader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    return acc

Files already downloaded and verified


FileNotFoundError: ignored

In [ ]:
model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
entry = removeModule(torch.load("/content/drive/MyDrive/Colab Notebooks/6787 Notebooks/Google Colab Scripts/baseOneShot/best.pth.tar")
# model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
# Load the file
model.load_state_dict(entry["state_dict"])
print(computeStateSizes(model.state_dict(), True))
print(computeStateSizes(model.state_dict(), False))
# entry = torch.load("../drive/MyDrive/CS6787 - Final Project/Google Colab Scripts/Pruning Models/Element Pruning/Direct Pruning/best.pth.tar", map_location=torch.device('cpu'))
startTime = time.time()
finalAcc = test(model)
finalTime = time.time() - startTime
stats = computeStatistics(model)
# stats["Name"] = entry.name
# stats["Time"] = finalTime
# results.append(stats)

# df = pd.DataFrame(results)

stats

3555175 3700128 15073560
14.237029075622559
3.39047908782959


RuntimeError: ignored

In [ ]:
print(baseParameters, baseTrainParameters, baseSize)

855770 855770 3.41556453704834


In [ ]:
modelDirectory = "test"

results = []
for entry in os.scandir(modelDirectory):
  model = distiller.models.create_model(False, "cifar10", "resnet56_cifar", True, None)
  #load the file
  torch.load(entry.path)
  model.load_state_dict(entry["state_dict"])
  startTime = time.time()
  finalAcc = test(model)
  finalTime = time.time() - startTime
  stats = computeStatistics(model)
  stats["Name"] = entry.name
  stats["Time"] = finalTime
  results.append(stats)

df = pd.DataFrame(results)

save_dir = "."
df.to_csv(save_dir)